Name - Shree Bohara, Github Username - ShreeBohara, USC id: 9969390417


In [1]:
import os
import pandas as pd
from scipy.stats import bootstrap
import numpy as np



# a
Data Downloaded

# b

In [2]:

base_path = "../data/AReM"

test_data = []
train_data = []

for activity in os.listdir(base_path):
    activity_path = os.path.join(base_path, activity)
    if os.path.isdir(activity_path):
        files = sorted(os.listdir(activity_path))
        
        for idx, filename in enumerate(files):
            file_path = os.path.join(activity_path, filename)
            

            df = pd.read_csv(
                file_path,
                comment="#",            # lines starting with '#' are treated as comments
                header=None,            # no header in the data lines
                names=["time", "avg_rss12", "var_rss12",
                       "avg_rss13", "var_rss13", "avg_rss23", "var_rss23"],
                sep=",",                
                skip_blank_lines=True,   
                on_bad_lines="skip",    
                engine="python"         
            )

            df.insert(0, "activity", activity)
            
            if activity in ["bending1", "bending2"]:
                if idx < 2:
                    test_data.append(df)
                else:
                    train_data.append(df)
            else:
                if idx < 3:
                    test_data.append(df)
                else:
                    train_data.append(df)

# Combine all the DataFrames
testdf = pd.concat(test_data, ignore_index=True)
traindf = pd.concat(train_data, ignore_index=True)

# Export to CSV
testdf.to_csv("test_data.csv", index=False)
traindf.to_csv("train_data.csv", index=False)



In [3]:
print("Test DataFrame info:")
testdf.info()

print("Train DataFrame info:")
traindf.info()


print("Test DataFrame shape:", testdf.shape)
print("Train DataFrame shape:", traindf.shape)


print("Test DataFrame head:")
print(testdf.head())

print("Train DataFrame head:")
print(traindf.head())


Test DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9120 entries, 0 to 9119
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   activity   9120 non-null   object 
 1   time       9120 non-null   int64  
 2   avg_rss12  9120 non-null   float64
 3   var_rss12  9120 non-null   float64
 4   avg_rss13  9120 non-null   float64
 5   var_rss13  9120 non-null   float64
 6   avg_rss23  9120 non-null   float64
 7   var_rss23  9120 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 570.1+ KB
Train DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33117 entries, 0 to 33116
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   activity   33117 non-null  object 
 1   time       33117 non-null  int64  
 2   avg_rss12  33117 non-null  float64
 3   var_rss12  33117 non-null  float64
 4   avg_rss13  33117 non-null  float64

# C

## i.


Time-domain features commonly used in time series classification include:

Mean,
Median,
Variance,
Standard deviation,
Minimum,
Maximum,
Range,
First quartile,
Third quartile,
Zero-crossings,
Skewness,
Kurtosis,
Energy,
Entropy,
Autocorrelation,
Trend,
Seasonality,









## ii.

In [4]:
time_series = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']

correct_columns = ['time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']

all_features = []

instance_number = 1
for activity in sorted(os.listdir(base_path)):
    activity_dir = os.path.join(base_path, activity)
    if os.path.isdir(activity_dir):
        
        for file_name in sorted(os.listdir(activity_dir)):
            if file_name.endswith('.csv'):
                file_path = os.path.join(activity_dir, file_name)
                try:
                    df = pd.read_csv(file_path, skiprows=5, header=None, on_bad_lines='skip')
                    df.columns = correct_columns
                    instance_features = []
                    instance_features.append(instance_number)
                    
                    for col in time_series:
                        series = df[col]
                        instance_features.extend([
                            np.min(series),
                            np.max(series),
                            np.mean(series),
                            np.median(series),
                            np.std(series),
                            np.percentile(series, 25),  
                            np.percentile(series, 75)   
                        ])
                    
                  
                    all_features.append(instance_features)
                    
                   
                    instance_number += 1
                except Exception as e:
                    print(f"Error processing file: {file_name} in {activity} folder. Error: {e}")

columns = ['Instance']  
for i in range(1, 7):
    columns.extend([
        f'min_{i}', f'max_{i}', f'mean_{i}', f'median_{i}', 
        f'std_{i}', f'1st_quart_{i}', f'3rd_quart_{i}'
    ])

features_df = pd.DataFrame(all_features, columns=columns)

print(features_df)


    Instance  min_1  max_1     mean_1  median_1     std_1  1st_quart_1  \
0          1  37.25  45.00  40.624792    40.500  1.475428        39.25   
1          2  38.00  45.67  42.812812    42.500  1.434054        42.00   
2          3  35.00  47.40  43.954500    44.330  1.557210        43.00   
3          4  33.00  47.75  42.179812    43.500  3.666840        39.15   
4          5  33.00  45.75  41.678063    41.750  2.241152        41.33   
..       ...    ...    ...        ...       ...       ...          ...   
83        84  20.75  46.25  34.763333    35.290  4.737266        31.67   
84        85  21.50  51.00  34.935812    35.500  4.641102        32.00   
85        86  18.33  47.67  34.333042    34.750  4.943612        31.25   
86        87  18.33  45.75  34.599875    35.125  4.726858        31.50   
87        88  15.50  43.67  34.225875    34.750  4.437168        31.25   

    3rd_quart_1  min_2  max_2  ...     std_5  1st_quart_5  3rd_quart_5  min_6  \
0       42.0000    0.0   1.30 

## iii.

In [5]:

std_devs = features_df.iloc[:, 1:].std()


def calc_std(data, axis=-1):
    return np.std(data, axis=axis)


bootstrap_intervals = {}

for col in features_df.columns[1:]:  # Skip the 'Instance' column
    data = features_df[col].values
    res = bootstrap((data,), calc_std, vectorized=True, confidence_level=0.90, n_resamples=10000)
    bootstrap_intervals[col] = (res.confidence_interval.low, res.confidence_interval.high)


std_df = pd.DataFrame({
    'Feature': features_df.columns[1:],
    'Std Dev': std_devs.values,
    'Lower Bound': [interval[0] for interval in bootstrap_intervals.values()],
    'Upper Bound': [interval[1] for interval in bootstrap_intervals.values()]
})

print("Standard Deviations and 90% Confidence Intervals:")
print(std_df)



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_resampling.py:144: RuntimeWarning: invalid value encountered in scalar divide
  a_hat = 1/6 * sum(nums) / sum(dens)**(3/2)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/_resampling.py:97: DegenerateDataWarning: The BCa confidence interval cannot be calculated. This problem is known to occur when the distribution is degenerate or the statistic is np.min.
  warnings.warn(DegenerateDataWarning(msg))


Standard Deviations and 90% Confidence Intervals:
        Feature   Std Dev  Lower Bound  Upper Bound
0         min_1  9.569975     8.488907    11.133816
1         max_1  4.394362     3.507361     5.479474
2        mean_1  5.335700     4.801922     5.982299
3      median_1  5.440054     4.881402     6.105590
4         std_1  1.770338     1.604359     1.988957
5   1st_quart_1  6.153874     5.641103     6.735184
6   3rd_quart_1  5.138925     4.438049     5.968769
7         min_2  0.000000          NaN          NaN
8         max_2  5.062729     4.682200     5.448451
9        mean_2  1.574198     1.410577     1.708444
10     median_2  1.412293     1.251581     1.550553
11        std_2  0.883215     0.806956     0.941281
12  1st_quart_2  0.946386     0.840519     1.041329
13  3rd_quart_2  2.125399     1.902557     2.298213
14        min_3  2.956462     2.784111     3.129535
15        max_3  4.875137     4.261770     5.543800
16       mean_3  4.008228     3.480724     4.540221
17     median_

## iv.


Mean: Captures the central tendency, giving an overall measure of the activity level.

Standard Deviation: Captures variability, distinguishing activities with different levels of consistency.

Maximum: Captures the peak value, indicating the highest intensity of the activity.





# 2

## a
The linear model is the correct model.
The cubic model has extra terms (𝑋2 and  𝑋3) that are not needed.

A model with more terms can always fit the training data as well as or better than a simpler model.

So, the cubic model will have a lower or equal training RSS compared to the linear model.

## b
The linear model is the right model and should work well on new (test) data.

The cubic model might fit the training data too closely, even capturing random noise.

This makes it worse at predicting new data (overfitting).
So, the cubic model will likely have a higher test RSS than the linear model.


## c
The linear model is too simple and cannot capture the full pattern in the data.

The cubic model has more flexibility to fit the data better.
More flexibility means a better fit on training data.

So, the cubic model will have a lower or equal training RSS compared to the linear model.

## d
If the true relationship is only slightly non-linear, the cubic model may overfit and do worse on test data.

If the true relationship is very non-linear, the cubic model might be a better choice and predict new data more accurately.

So, the test RSS for the cubic model could be lower or higher than the linear model, depending on how much non-linearity is present.

## References
https://dganesan.github.io/mhealth-course/chapter3-activityrecognition/ch3-time-domain-features.html